# Machine learning methods

This notebook demonstrates basic methods for machine learning on tabular data. Need python modules scikit-learn, sweetviz, lightgbm, optuna and tpot.

## Import libraries

In [ ]:
import pandas as pd
import os
import pickle
import re
import numpy as np
import matplotlib.pyplot as plt

#pandas api
from pandas.api.types import is_string_dtype, is_object_dtype

#encoders and preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

#feature engineering
from sklearn.preprocessing import MinMaxScaler

#feature importance, feature selection
from sklearn.inspection import permutation_importance

#exploratory data analysis
import sweetviz
from sweetviz.feature_config import FeatureConfig

#models
from lightgbm import LGBMClassifier, LGBMRegressor #sudo apt install libgomp1
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression

#model interpretation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import ConfusionMatrixDisplay, PredictionErrorDisplay

#model optimization
import optuna
import optuna.visualization as vis

#autoML
from tpot import TPOTClassifier, TPOTRegressor


In [ ]:
def regression_report(y_true, y_pred, output_dict=True):
    """Generates a regression report similar to classification_report in sklearn.metrics.
    The explained variance, r2 score, mean absolute error (MAE), mean squared error (MSE) and square root mean squared error (RMSE) are reported.

    :param y_true: Ground truth (correct) target values.
    :type y_true: array, shape = [n_samples]
    :param y_pred: Estimated targets as returned by model.
    :type y_pred: array, shape = [n_samples]
    :param output_dict: Return a dictionary of reported scores, defaults to True.
    :type output_dict: bool, optional
    :return: Regression report
    :rtype: dict
    """    

    import sklearn.metrics as metrics
    import numpy as np

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    #mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    #median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    if output_dict is True:
        report = {
            'explained_variance' : round(explained_variance,4),
            #'mean_squared_log_error' : round(mean_squared_log_error,4),
            'R2' : round(r2,4),
            'MAE' : round(mean_absolute_error,4),
            'MSE' : round(mse,4),
            'RMSE': round(np.sqrt(mse),4)
        }
        return report
    else:
        print('explained_variance: ', round(explained_variance,4))    
        #print('mean_squared_log_error: ', round(mean_squared_log_error,4))
        print('R2: ', round(r2,4))
        print('MAE: ', round(mean_absolute_error,4))
        print('MSE: ', round(mse,4))
        print('RMSE: ', round(np.sqrt(mse),4))

## Read csv
(or load through databricks-connect SparkSQL session)

In [ ]:
df = pd.read_csv('stat_med_acs.csv').sort_values(by='value').reset_index(drop=True)
df.head()

Correct column names, eliminate special characters and spaces

In [ ]:
new_colnames = {c: re.sub(r"[^a-zA-Z0-9]+", ' ', c).replace(' ', '_') for c in df.columns.tolist()}

df = df.rename(columns=new_colnames)
df.head()

## Encoders

### 1. Label Encoder

Used for encode target labels for classification to monotonically increasing numeric labels.

In [ ]:
enc = LabelEncoder()

enc.fit(df['stage'])

enc.classes_

    !! Note that the order of classes don't reflect severity of kidney failure

In [ ]:
from sklearn.utils import column_or_1d

# Custom encoder class
class MyLabelEncoder(LabelEncoder):

    # Override fit method
    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self

In [ ]:
myenc = MyLabelEncoder()

myenc.fit(df['stage'])

myenc.classes_

In [ ]:
df['stage'] = myenc.transform(df['stage'])
df.iloc[:,:8].head()

### 2. Ordinal Encoder

Similar to Label Encoder with handling of unknown and missing values in transformed data

Note: expects 2-dimensional data

In [ ]:
enc = OrdinalEncoder()

df['sex_encoded'] = enc.fit_transform(df['sex'].to_numpy().reshape(-1, 1))
df.head()

### 3. One-Hot Encoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

transformed = enc.fit_transform(df['sex'].to_numpy().reshape(-1, 1))
enc.get_feature_names_out()

In [ ]:
feature_names = enc.get_feature_names_out()
for i in range(len(feature_names)):
    if feature_names[i].endswith('nan') or feature_names[i].endswith('None') or feature_names[i].endswith('_U'):
        continue
    f = '_'.join(['sex'] + feature_names[i].split('_')[1:])
    df[f] = transformed.todense()[:,i]

df.head()

Finalize data

In [ ]:
df = df.drop(columns=['patient_id', 'sex_encoded', 'sex_F', 'sex_M'], axis=1, errors='ignore')

enc = LabelEncoder()
df['sex'] = enc.fit_transform(df['sex'])
df.head()

## Split data to train and test set

In [ ]:
# Reduce data for faster computations
_, df_reduced = train_test_split(df, test_size=0.2*0.2, stratify=df['stage'], random_state=42)

# Split data
X_train, X_test = train_test_split(df_reduced, test_size=0.2, stratify=df_reduced['stage'], random_state=42)

# Make regression data
Xr_train = X_train.copy().drop(columns=['stage'], axis=1)
Xr_test  = X_test.copy().drop(columns=['stage'], axis=1)
Xr_train.to_csv('stat_med_acs_regr_train.csv', index=False)
Xr_train.to_csv('stat_med_acs_regr_test.csv', index=False)

yr_train = Xr_train['value']
Xr_train = Xr_train.drop(columns=['value'], axis=1)
yr_test = Xr_test['value']
Xr_test = Xr_test.drop(columns=['value'], axis=1)

# Make classification data
Xc_train = X_train.copy().drop(columns=['value'], axis=1)
Xc_test  = X_test.copy().drop(columns=['value'], axis=1)
Xc_train.to_csv('stat_med_acs_clas_train.csv', index=False)
Xc_train.to_csv('stat_med_acs_clas_test.csv', index=False)

yc_train = Xc_train['stage']
Xc_train = Xc_train.drop(columns=['stage'], axis=1)
yc_test = Xc_test['stage']
Xc_test = Xc_test.drop(columns=['stage'], axis=1)

## Feature importance

**Feature importance** provides predicted measures to compare the influence of features on a specific modeling task. Some of the more popular techniques:
1. **Permutated feature importance**, which uses an arbitrary regression or classification modeling algorithm, and measures its performance every time we permutate a single feature to compare performance between models with the original feature and the permutated one. Permutating every feature multiple times allows to predict their influence in the current task accuartely.
2. **SHAP values**

In [ ]:
from sklearn.metrics import get_scorer_names
get_scorer_names()

Plotting function

In [ ]:
def plotFI(features, importance, target='', modelname='', scorer='', n=20):
    from matplotlib import pyplot

    if n is not None and isinstance(n, int):
        features = features[:n]
        importance = importance[:n]
    
    importance = [round(fi, ndigits=2) for fi in importance]

    fig, ax = pyplot.subplots(layout='constrained')
    bars = ax.bar([x for x in range(len(importance))], importance)
    ax.bar_label(bars, padding=3, rotation=75)
    ax.set_title('Feature importance (target: {0})\n{1} - {2}'.format(target, modelname, scorer))
    ax.set_xticks(np.arange(len(features)), features)
    ax.set_ylim(0, np.max(importance)*1.2)
    pyplot.xticks(rotation=90)
    pyplot.show()

Regression model

In [ ]:
# Set up model to fit adta with
model = LGBMRegressor()
scorer = 'neg_root_mean_squared_error'

# Fit model
model.fit(Xr_train, yr_train)

# Fit model with permutated data to assess permutation-based feature importance
FI = permutation_importance(model, Xr_train, yr_train, scoring=scorer, n_repeats=5)

# Sort importances
importance = FI.importances_mean
order = sorted(range(len(importance)), key=lambda k: importance[k], reverse=True)

importance_order = [importance[i] for i in order]
features_order = [Xr_train.columns[i] for i in order]

# Plot feature importance
plotFI( features_order, importance_order, 'value', model.__class__.__name__, scorer )

Classification model

In [ ]:
# Set up model to fit adta with
model = LGBMClassifier()
scorer = 'neg_log_loss'

# Fit model
model.fit(Xc_train, yc_train)

# Fit model with permutated data to assess permutation-based feature importance
FI = permutation_importance(model, Xc_train, yc_train, scoring=scorer, n_repeats=5)

# Sort importances
importance = FI.importances_mean
order = sorted(range(len(importance)), key=lambda k: importance[k], reverse=True)

importance_order = [importance[i] for i in order]
features_order = [Xc_train.columns[i] for i in order]

# Plot feature importance
plotFI( features_order, importance_order, 'stage', model.__class__.__name__, scorer )

## Feature selection

Feature selection is the process of selecting a subset of relevant features (variables, predictors) for use in model construction (<a href="https://en.wikipedia.org/wiki/Feature_selection">Wikipedia</a>).

Collection of various feature selection methods: https://github.com/mlpapers/feature-selection

In this example we select the top 20 features in feature importance calculations.

In [ ]:
Xc_train = Xc_train[features_order[0:20]]
Xc_test = Xc_test[features_order[0:20]]

## Exploratory data analysis

Explorative data analysis provides data visualization for the following purposes:
- View distribution of variables
- Review data quality (missing values, data type)
- Understand associations between variables used as features

We use SweetViz for EDA (https://github.com/fbdesignpro/sweetviz)

In [ ]:
Xc_train['stage'] = yc_train
Xc_test['stage'] = yc_test

# Force target variable to be numeric
sweetviz_config = FeatureConfig( force_num=['stage'] )

# One set EDA
#sweetviz_report = sweetviz.analyze([Xc_train, 'medication train.']
#                                   target_feat='stage', feat_cfg=sweetviz_config, pairwise_analysis='on')

# Two set comparison
sweetviz_report = sweetviz.compare([Xc_train, 'medication train.'], [Xc_test, 'mediacation test'],
                                   target_feat='stage', feat_cfg=sweetviz_config, pairwise_analysis='on')

Report EDA

In [ ]:
#sweetviz_report.show_notebook(w='100%')

## Machine learning models

In [ ]:
Xc_train = Xc_train.drop(columns=['stage'], axis=1)
Xc_test = Xc_test.drop(columns=['stage'], axis=1)

### 1. Logistic Regression

In [ ]:
# Set up model
model_lrg = LogisticRegression()

# Fit model
model_lrg.fit(Xc_train, yc_train)

# Predict 'stage' on test set
yc_pred = model_lrg.predict(Xc_test)

# Report classification metrics
report = classification_report(yc_test, yc_pred, output_dict=True)
print('Accuracy: ' + str(report['accuracy']))
print(report['weighted avg'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    model_lrg, Xc_test, yc_test, display_labels=myenc.classes_.tolist(), cmap=plt.cm.Blues, normalize=None
)
disp.ax_.set_title("Confusion matrix")
plt.show()

### 2. Random Forest Classifier

In [ ]:
# Set up model
model_rfc = RandomForestClassifier()

# Fit model
model_rfc.fit(Xc_train, yc_train)

# Predict 'stage' on test set
yc_pred = model_rfc.predict(Xc_test)

# Report classification metrics
report = classification_report(yc_test, yc_pred, output_dict=True)
print('Accuracy: ' + str(report['accuracy']))
print(report['weighted avg'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    model_rfc, Xc_test, yc_test, display_labels=myenc.classes_.tolist(), cmap=plt.cm.Blues, normalize=None
)
disp.ax_.set_title("Confusion matrix")
plt.show()

# Feature importance of fitted model
plotFI( model_rfc.feature_names_in_, model_rfc.feature_importances_, target='stage', modelname=model_rfc.__class__.__name__ )

### 3. Gradient Boosting Classifier (LightGBM)

In [ ]:
# Set up model
model_lgb = LGBMClassifier()

# Fit model
model_lgb.fit(Xc_train, yc_train)

# Predict 'stage' on test set
yc_pred = model_lgb.predict(Xc_test)

# Report classification metrics
report = classification_report(yc_test, yc_pred, output_dict=True)
print('Accuracy: ' + str(report['accuracy']))
print(report['weighted avg'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    model_lgb, Xc_test, yc_test, display_labels=myenc.classes_.tolist(), cmap=plt.cm.Blues, normalize=None
)
disp.ax_.set_title("Confusion matrix")
plt.show()

# Feature importance of fitted model
plotFI( model_lgb.feature_name_, model_lgb.feature_importances_, target='stage', modelname=model_lgb.__class__.__name__ )

### 4. Regression example

In [ ]:
# Set up model
model_lgbr = LGBMRegressor()

# Fit model
model_lgbr.fit(Xr_train, yr_train)

# Predict 'stage' on test set
yr_pred = model_lgbr.predict(Xr_test)

# Report classification metrics
report_regr = regression_report(yr_test, yr_pred, output_dict=True)
print(report_regr)

# Plot regression error
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    yr_test,
    y_pred=yr_pred,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(
    yr_test,
    y_pred=yr_pred,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
fig.suptitle("Plotting regression predictions")
plt.tight_layout()
plt.show()

# Feature importance of fitted model
plotFI( model_lgbr.feature_name_, model_lgbr.feature_importances_, target='stage', modelname=model_lgbr.__class__.__name__ )

## Optimization of models

Using the **Optuna** hyperparameter optimization framework, we can optimize the model parameters for better fit. This needs definition of objective function to minimize (or maximize, based on the scoring method), with suggestions on parameter ranges.

### 1. Logistic Regression

Objective function

In [ ]:
def objective_lrc(trial):
    # Suggest values for hyperparameters
    penalty = trial.suggest_categorical('penalty', ['l2', 'l1'])
    if penalty == 'l1':
        solver = 'saga'
    else:
        solver = 'lbfgs'
    regularization = trial.suggest_uniform('logistic-regularization', 0.01, 10)

    # Create and fit model
    model = LogisticRegression(
        penalty=penalty,
        C=regularization,
        solver=solver,
        random_state=42,
    )
    model.fit(Xc_train, yc_train)

    # Make predictions and calculate score
    yc_pred = model.predict(Xc_test)
    acc = accuracy_score(yc_test, yc_pred)

    # Return Accuracy
    return acc

### 2. Random Forest Classifier

Objective function

In [ ]:
def objective_rfc(trial):
    # Suggest values for hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Create and fit model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(Xc_train, yc_train)

    # Make predictions and calculate score
    yc_pred = model.predict(Xc_test)
    acc = accuracy_score(yc_test, yc_pred)

    # Return Accuracy
    return acc

Create optimization study

In [ ]:
# Create study object
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())

# Run optimization process
study.optimize(objective_rfc, n_trials=20, show_progress_bar=True)

Visualize process

In [ ]:
# Plot optimization history
vis.plot_optimization_history(study)

In [ ]:
# Plot parameter importance
vis.plot_param_importances(study)

In [ ]:
# Plot slice plot
vis.plot_slice(study, params=["n_estimators", "max_depth"])

In [ ]:
# Plot contour plot
vis.plot_contour(study, params=["min_samples_split", "min_samples_leaf"])

In [ ]:
# Plot parallel_coordinate
vis.plot_parallel_coordinate(study)

In [ ]:
# Print best trial and best hyperparameters
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Train model with best hyperparameters

In [ ]:
# Set up model
model_rfc = RandomForestClassifier(**study.best_params)

# Fit model
model_rfc.fit(Xc_train, yc_train)

# Predict 'stage' on test set
yc_pred = model_rfc.predict(Xc_test)

# Report classification metrics
report = classification_report(yc_test, yc_pred, output_dict=True)
print('Accuracy: ' + str(report['accuracy']))
print(report['weighted avg'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    model_rfc, Xc_test, yc_test, display_labels=myenc.classes_.tolist(), cmap=plt.cm.Blues, normalize=None
)
disp.ax_.set_title("Confusion matrix")
plt.show()

# Feature importance of fitted model
plotFI( model_rfc.feature_names_in_, model_rfc.feature_importances_, target='stage', modelname=model_rfc.__class__.__name__ )

In [ ]:
model_rfc

### 3. Gradient Boosting Classifier (LightGBM)

In [ ]:
def objective_lgbc(trial):
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }
    
    model = LGBMClassifier(**param)

    model.fit(Xc_train, yc_train)

    # Make predictions and calculate score
    yc_pred = model.predict(Xc_test)
    acc = accuracy_score(yc_test, yc_pred)

    # Return Accuracy
    return acc

### 4. Regression objective function examples

In [ ]:
def objective_rfr(trial):
    # Suggest values for hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Create and fit model
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(Xr_train, yr_train)

    # Make predictions and calculate RMSE
    yr_pred = model.predict(Xr_test)
    rmse = np.sqrt(mean_squared_error(yr_test, yr_pred))
    mae = mean_absolute_error(yr_test, yr_pred)
    r2 = r2_score(yr_test, yr_pred)

    # Return RMSE
    return rmse

In [ ]:
def objective_lgbr(trial):
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    
    model = LGBMClassifier(**param)

    model.fit(Xr_train, yr_train)

    # Make predictions and calculate RMSE
    yr_pred = model.predict(Xr_test)
    rmse = np.sqrt(mean_squared_error(yr_test, yr_pred))
    mae = mean_absolute_error(yr_test, yr_pred)
    r2 = r2_score(yr_test, yr_pred)

    # Return RMSE
    return rmse

## Automated Machine Learning

with TPOT

In [ ]:
# Set up TPOT classifier
model = TPOTClassifier(
    max_time_mins=15,
    cv=5,
    scoring='neg_log_loss',
    random_state=42,
    verbosity=2
    )

# Optimize and fit pipelines
model.fit(Xc_train, yc_train)

# Export best pipeline and fitted model
model.export('stat_med_acs_clas_pipeline.txt')
pickle.dump( model.fitted_pipeline_, open( 'stat_med_acs_clas_model.pkl', "wb" ) )

In [ ]:
# Predict 'stage' on test set
yc_pred = model.predict(Xc_test)

# Report classification metrics
report = classification_report(yc_test, yc_pred, output_dict=True)
print('Accuracy: ' + str(report['accuracy']))
print(report['weighted avg'])

# Plot confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
    model.fitted_pipeline_, Xc_test, yc_test, display_labels=myenc.classes_.tolist(), cmap=plt.cm.Blues, normalize=None
)
disp.ax_.set_title("Confusion matrix")
plt.show()